In [31]:
# setting path etc.
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

# Path: /Users/harshvardhan/Library/CloudStorage/Dropbox/Academics/UTK Classes/Spring 2023/Deep Learning/Final Project - FairFace Dataset

In [32]:
from sklearn.preprocessing import MinMaxScaler
import keras
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from PIL import Image
from concurrent.futures import ThreadPoolExecutor
import os
import time
import random

In [33]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import keras
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import argparse
import os
import seaborn as sns
from tensorflow.keras.layers import Lambda, Input, Dense,Conv2DTranspose,Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.losses import mse, binary_crossentropy
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K
import matplotlib.gridspec as gridspec
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, Lambda, Reshape, Conv2DTranspose

# Function for one-hot encoding gender, age and race

In [34]:
def convert_age(age):
    age_dict = {'0-2': 0, '3-9': 1, '10-19': 2, '20-29': 3, '30-39': 4, '40-49': 5, '50-59': 6, '60-69': 7, 'more than 70': 8}
    return np.eye(9)[[age_dict[a] for a in age]]

def convert_gender(gender):
    gender_dict = {'Male': 0, 'Female': 1}
    return np.eye(2)[[gender_dict[g] for g in gender]]

def convert_race(race):
    race_dict = {'Black': 0, 'Latino_Hispanic': 1, 'East Asian': 2, 'White': 3, 'Southeast Asian': 4, 'Middle Eastern': 5, 'Indian': 6}
    return np.eye(7)[[race_dict[r] for r in race]]

In [35]:
df = pd.read_csv('fairface-img-margin025-trainval/fairface_label_train.csv')
df.shape

(86744, 5)

In [36]:
# function to read image
def read_image(image_path):
    with Image.open(image_path) as image:
        return np.asarray(image)

# function to read image
def read_image(image_path):
    with Image.open(image_path) as image:
        return np.asarray(image)

# read data, labels in lists
def get_dataset(DATA_DIR, mode, sample=False):
    if mode == 'train':
        df = pd.read_csv(os.path.join(DATA_DIR, 'fairface_label_train.csv'))
    elif mode == 'val':
        df = pd.read_csv(os.path.join(DATA_DIR, 'fairface_label_val.csv'))
    else:
        raise ValueError

    age = df['age'].values.tolist()
    gender = df['gender'].values.tolist()
    race = df['race'].values.tolist()
    filenames = df['file'].values.tolist()

    image_paths = [os.path.join(DATA_DIR, name) for name in filenames]

    if sample:
        sample_size = int(len(image_paths) * 0.05) # 5% of total images
        sampled_indexes = random.sample(range(len(image_paths)), sample_size)
        image_paths = [image_paths[i] for i in sampled_indexes]
        age = [age[i] for i in sampled_indexes]
        gender = [gender[i] for i in sampled_indexes]
        race = [race[i] for i in sampled_indexes]

    with ThreadPoolExecutor() as executor:
        all_img = list(executor.map(read_image, image_paths))
    
    onehot_age = convert_age(age)
    onehot_gender = convert_gender(gender)
    onehot_race = convert_race(race)

    return all_img, onehot_age, onehot_gender, onehot_race

In [37]:
DATA_DIR = '/Users/harshvardhan/Library/CloudStorage/Dropbox/Academics/UTK Classes/Spring 2023/Deep Learning/Final Project - FairFace Data/cosc-525-final-project/fairface-img-margin025-trainval'

In [40]:
train_img, train_age, train_gender, train_race = get_dataset(DATA_DIR,'train', sample=True)
val_img, val_age, val_gender, val_race = get_dataset(DATA_DIR,'val', sample=True)

# min-max scaling
flattened_train_img = [img.reshape(224*224*3) for img in train_img]
flattened_val_img = [img.reshape(224*224*3) for img in val_img]

scaler = MinMaxScaler()
scaler.fit(flattened_train_img)
print(len(scaler.data_max_), len(scaler.data_min_))

scaled_train_img = scaler.transform(flattened_train_img)
scaled_val_img = scaler.transform(flattened_val_img)

scaled_train_img = np.array([img.reshape(224,224,3) for img in scaled_train_img])
scaled_val_img = np.array([img.reshape(224,224, ) for img in scaled_val_img])

150528 150528


ValueError: cannot reshape array of size 150528 into shape (224,224)

# Auto Encoder

In [ ]:
def sampling(args):
    """Reparameterization trick by sampling from an isotropic unit Gaussian.

    # Arguments
        args (tensor): mean and log of variance of Q(z|X)

    # Returns
        z (tensor): sampled latent vector
    """
    #Extract mean and log of variance
    z_mean, z_log_var = args
    #get batch size and length of vector (size of latent space)
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]

    # by default, random_normal has mean = 0 and std = 1.0
    epsilon = K.random_normal(shape=(batch, dim))
    #Return sampled number (need to raise var to correct power)
    return z_mean + K.exp(z_log_var) * epsilon

In [ ]:
latent_dim = 10

# Decoder Model

In [ ]:
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')

x = Dense(128, activation='relu', name="decoder_hidden_layer")(latent_inputs)
x = Reshape((2,2,32))(x)
x = Conv2DTranspose(filters = 64, kernel_size = (3,3), strides = 2, padding = 'same',activation = 'relu')(x)
x = Conv2DTranspose(filters=32,kernel_size=3,strides=2,padding='valid', activation='relu')(x)
x = Conv2D(filters=16,kernel_size=3,strides=1,padding='valid', activation='relu')(x)
x = Conv2DTranspose(filters=16,kernel_size=3,strides=2,padding='valid', activation='relu')(x)
x = Conv2DTranspose(filters=1,kernel_size=4,strides=2,padding='valid', activation='relu')(x)

In [ ]:
# instantiate decoder model
decoder = Model(latent_inputs, outputs=x, name='decoder_output')
decoder.summary()